In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
import re
import pickle


In [4]:
df = pd.read_excel("../data/raw/Online_Retail.xlsx")

In [5]:
df.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [6]:
df.shape

(541909, 8)

In [7]:
# find null values --> and remove
for col in df.columns:
    print(f"{col}: {df[col].isnull().sum()}")

InvoiceNo: 0
StockCode: 0
Description: 1454
Quantity: 0
InvoiceDate: 0
UnitPrice: 0
CustomerID: 135080
Country: 0


In [8]:
# Drop the rows with null values
df = df.dropna()

df.drop_duplicates(inplace = True)

df.shape

(401604, 8)

In [9]:
# unit price, remove 0 to avoid confusion
df = df[df['UnitPrice'] > 0]

Stockcodes

In [10]:
unique_stock_codes = df['StockCode'].unique()

# Function to check if a code is "weird" (contains 0 or 1 digit)
def is_weird(code):
    return sum(c.isdigit() for c in str(code)) in (0, 1)

# Filter the unique stock codes to find the "weird" ones
anomalous_stock_codes = [code for code in unique_stock_codes if is_weird(code)]

# Printing each "weird" stock code and its frequency
print("Weird stock codes:")
for code in anomalous_stock_codes:
    frequency = (df['StockCode'] == code).sum()
    print(f"{code}: {frequency}")


Weird stock codes:
POST: 1196
D: 77
C2: 134
M: 454
BANK CHARGES: 12
PADS: 3
DOT: 16
CRUK: 16


In [11]:
# remove weird stock codes
df = df[~df['StockCode'].isin(anomalous_stock_codes)]
df.reset_index(drop=True, inplace=True)
df.shape

(399656, 8)

Description

In [12]:
# Function to remove color words from the description, for better cluster
def remove_color_words(description):
    color_words = ['red', 'blue', 'green', 'yellow', 'orange', 'purple', 'pink', 'brown', 'black', 'white','cream']
    words = description.split()
    filtered_words = [word for word in words if word.lower() not in color_words]
    cleaned_description = ' '.join(filtered_words)
    return cleaned_description

df['Description'] = df['Description'].astype(str)

# Apply the remove_color_words function
df['Description'] = df['Description'].apply(remove_color_words)

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,WOOLLY HOTTIE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


Pickle

In [13]:
# upload to the interim 
df.to_pickle("../data/interim/01_data_preprocessed.pkl")